# Palabras clave

## Palabras clave. 1ª versión

In [1]:
from collections import defaultdict

def get_keywords(words):
    word_counts = defaultdict(lambda : 0)
    for w in words:
        word_counts[w]+= 1
    return [ x for x,_ in sorted(word_counts.iteritems(), key=lambda (_,freq):freq, reverse=True)][:10]

### Carga de documentos

Dataset URL: http://www.lsi.us.es/~fermin/corpusCine.zip

Nos quedamos con los ficheros .xml, que tienen una estructura del estilo:

```xml
<review author="XXX" title="XXX" rank="X" maxRank="X" source="XXX">
	<summary>XXX</summary>
	<body>XXX</body>
</review>
```

In [2]:
from bs4 import BeautifulSoup
import os
import config

In [3]:
import codecs  # Know your encoding
import re

def parse_file(file_path):
    retval = {'file': os.path.basename(file_path)}
    with codecs.open(file_path,'r', encoding='ISO-8859-15') as fd:
        f = fd.read().strip()
    reg_expr = re.compile(r'\<review author="(?P<author>.*)" title="(?P<title>.*)" rank="(?P<rank>\d)".*\>\s*<summary>(?P<summary>.*)</summary>\s*<body>(?P<body>.*)</body>\s*</review>')
    regexp_result = reg_expr.search(f.strip())
    for key in ['author', 'rank', 'title', 'summary', 'body']:
        retval[key] = regexp_result.group(key)
    return retval

In [5]:
documents = []
for file_name in os.listdir(config.DATASET_MUCHOCINE_RAW):
    try:
        documents.append(
            parse_file(config.DATASET_MUCHOCINE_RAW+'/'+file_name)
        )
    except Exception as e:
        print file_name
        print e.message
documents.sort(key=lambda x: x['file'])

In [6]:
print documents[0]

{'body': u'"May, \xbfQuieres ser mi amigo?" es una de esas pel\xedculas que nos recuerdan que el terror no siempre lleva garras de acero en una mano o una mascara en la cara. El terror y la locura se encuentran mucho m\xe1s cerca de nosotros, de la realidad, de nuestra pac\xedfica y hasta a veces aburrida monoton\xeda. May funciona bajo el m\xe9todo: la bestia duerme dentro de nosotros. En cada uno de nosotros hay un posible psic\xf3pata y nuestra vecina la del segundo puede ocultar un oscuro pasado o una doble vida. Para ello, Lucky McKee, nos narra efectivamente una historia presuntamente cotidiana (que sin embargo, engancha desde el principio) sobre una joven muy parecida a esas otras tantas que pululan a nuestro alrededor. La rarita de la clase, la ni\xf1a t\xedmida de la tienda de comestibles, esa extra\xf1a hermana de nuestro amigo, nuestra prima la del pueblo. Es por esto que nos montamos en un tren de cercan\xedas y no en un tren de alta velocidad con escenas m\xe1s vistas que 

### Cuenta palabras

In [7]:
def txt2words(txt):
    words = txt.split(' ')
    return words

In [8]:
print get_keywords(txt2words(documents[0]['body']))

[u'de', u'y', u'que', u'la', u'una', u'un', u'en', u'por', u'nos', u'no']


## 2ª versión. Stopwords

In [9]:
## Taken from NLTK corpus so you don't have to download

#import nltk
#nltk.download()
#from nltk.corpus import stopwords
#print filter_symbols(
#    ' '.join(
#        sorted(stopwords.words('spanish'))
#    )
#)

STOPWORDS = set('''
a al algo algunas algunos ante antes como con contra cual cuando de del desde donde durante e el ella ellas ellos
en entre era erais eran eras eres es esa esas ese eso esos esta estaba estabais estaban estabas estad estada estadas
estado estados estamos estando estar estaremos estara estaran estaras estare estareis estaria estariais estariamos
estarian estarias estas este estemos esto estos estoy estuve estuviera estuvierais estuvieran estuvieras estuvieron
estuviese estuvieseis estuviesen estuvieses estuvimos estuviste estuvisteis estuvieramos estuviesemos estuvo esta
estabamos estais estan estas este esteis esten estes fue fuera fuerais fueran fueras fueron fuese fueseis fuesen fueses
fui fuimos fuiste fuisteis fueramos fuesemos ha habida habidas habido habidos habiendo habremos habra habran habras
habre habreis habria habriais habriamos habrian habrias habeis habia habiais habiamos habian habias han has hasta
hay haya hayamos hayan hayas hayais he hemos hube hubiera hubierais hubieran hubieras hubieron hubiese hubieseis
hubiesen hubieses hubimos hubiste hubisteis hubieramos hubiesemos hubo la las le les lo los me mi mis mucho muchos
muy mas mi mia mias mio mios nada ni no nos nosotras nosotros nuestra nuestras nuestro nuestros o os otra otras otro
otros para pero poco por porque que quien quienes que se sea seamos sean seas sentid sentida sentidas sentido sentidos
seremos sera seran seras sere sereis seria seriais seriamos serian serias seais siente sin sintiendo sobre sois somos
son soy su sus suya suyas suyo suyos si tambien tanto te tendremos tendra tendran tendras tendre tendreis tendria
tendriais tendriamos tendrian tendrias tened tenemos tenga tengamos tengan tengas tengo tengais tenida tenidas tenido
tenidos teniendo teneis tenia teniais teniamos tenian tenias ti tiene tienen tienes todo todos tu tus tuve tuviera
tuvierais tuvieran tuvieras tuvieron tuviese tuvieseis tuviesen tuvieses tuvimos tuviste tuvisteis tuvieramos tuviesemos
tuvo tuya tuyas tuyo tuyos tu un una uno unos vosostras vosostros vuestra vuestras vuestro vuestros y ya yo el eramos
'''.split())

def txt2words(txt):
    words = [w for w in txt.split(' ') if w not in STOPWORDS]
    return words

In [10]:
print get_keywords(txt2words(documents[0]['body']))

[u'm\xe1s', u'La', u'En', u'terror', u'pel\xedcula', u'tren', u'May', u'ni\xf1a', u'adem\xe1s', u'bien']


## 3ª versión. Quitando mayúsculas, tildes...

In [11]:
def txt2words(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [w for w in txt.split(' ') if w not in STOPWORDS]
    return words

In [12]:
print get_keywords(txt2words(documents[0]['body']))

[u'may', u'pelicula', u'terror', u'tren', u'aquella', u'solondz', u'ni\xf1a', u'bien', u'cruda', u'cara']


## 4ª versión. Raíces léxicas

In [13]:
from nltk.stem.snowball import SpanishStemmer
stemmer = SpanishStemmer()

def txt2words(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [
        stemmer.stem(w)
        for w in txt.split(' ')
        if w not in STOPWORDS
    ]
    return words

In [14]:
print get_keywords(txt2words(documents[0]['body']))

[u'may', u'pelicul', u'personaj', u'terror', u'efect', u'tren', u'amig', u'negr', u'solondz', u'crud']


## Para todos los documentos

In [16]:
for d in documents:
    for key in ['body', 'summary']:
        d[key+'_tokens'] = txt2words(d[key])

## Exportar/Importar

In [ ]:
import json
import zlib


# Keys to keep
keys_export = ['file', 'title', 'author','rank', 'summary_tokens', 'body_tokens']
# New data structure to export
docs_export = [
    {key: d[key] for key in keys_export}
    for d in documents
]

# Export
with open(config.DATASET_MUCHOCINE, 'w+') as fd:
    fd.write(zlib.compress(json.dumps(docs_export)))
    
# Import
with open(config.DATASET_MUCHOCINE, 'r') as fd:
    docs_import =json.loads(zlib.decompress(fd.read()))